In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
df=pd.read_csv('cardekho_dataset.csv')
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


# Data Cleaning

### 1. Handling missing values

In [4]:
# Check null values
df.isnull().sum()

Unnamed: 0           0
car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [5]:
# Remove unnecessary columns
df.drop('car_name', axis=1, inplace=True)
df.drop('brand', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df['model'].unique()

array(['Alto', 'Grand', 'i20', 'Ecosport', 'Wagon R', 'i10', 'Venue',
       'Swift', 'Verna', 'Duster', 'Cooper', 'Ciaz', 'C-Class', 'Innova',
       'Baleno', 'Swift Dzire', 'Vento', 'Creta', 'City', 'Bolero',
       'Fortuner', 'KWID', 'Amaze', 'Santro', 'XUV500', 'KUV100', 'Ignis',
       'RediGO', 'Scorpio', 'Marazzo', 'Aspire', 'Figo', 'Vitara',
       'Tiago', 'Polo', 'Seltos', 'Celerio', 'GO', '5', 'CR-V',
       'Endeavour', 'KUV', 'Jazz', '3', 'A4', 'Tigor', 'Ertiga', 'Safari',
       'Thar', 'Hexa', 'Rover', 'Eeco', 'A6', 'E-Class', 'Q7', 'Z4', '6',
       'XF', 'X5', 'Hector', 'Civic', 'D-Max', 'Cayenne', 'X1', 'Rapid',
       'Freestyle', 'Superb', 'Nexon', 'XUV300', 'Dzire VXI', 'S90',
       'WR-V', 'XL6', 'Triber', 'ES', 'Wrangler', 'Camry', 'Elantra',
       'Yaris', 'GL-Class', '7', 'S-Presso', 'Dzire LXI', 'Aura', 'XC',
       'Ghibli', 'Continental', 'CR', 'Kicks', 'S-Class', 'Tucson',
       'Harrier', 'X3', 'Octavia', 'Compass', 'CLS', 'redi-GO', 'Glanza',
       

In [6]:
num_features = [feature for feature in df.columns if df[feature].dtype != 'O']
print('Num of numerical features : ', len(num_features))
cat_features = [feature for feature in df.columns if df[feature].dtype == 'O']
print('Num of categorical features : ', len(cat_features))
discrete_features=[feature for feature in num_features if len(df[feature].unique())<=25]
print('Num of discrete features : ', len(discrete_features))
continuous_features=[feature for feature in num_features if feature not in discrete_features]
print('Num of continuous features : ', len(continuous_features))

Num of numerical features :  7
Num of categorical features :  4
Num of discrete features :  2
Num of continuous features :  5


In [7]:
# Independent and dependent features
from sklearn.model_selection import train_test_split
X =df.drop(['selling_price'], axis=1)
y= df['selling_price']

### Feature Encoding and Scalling

In [8]:
len(df['model'].unique())

120

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['model'] = le.fit_transform(X['model'])

In [10]:
# Create column transformer with 2 types of transformers
num_features = X.select_dtypes(exclude='object').columns
onehot_columns = ['seller_type','fuel_type','transmission_type']
label_encoder_columns = ['model']

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer([
    ("OneHotEncoder", oh_transformer, cat_features),
    ('StandardScaler', numeric_transformer, num_features)],remainder='passthrough')

In [11]:
X=preprocessor.fit_transform(X)

In [12]:
# Separate dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# Model Training and model selection

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [14]:
# Create a function to evaluate model
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [15]:
models={
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')

    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))

    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 344702.2386
- Mean Absolute Error: 167198.2437
- R2 Score: 0.8535
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 396964.8801
- Mean Absolute Error: 179447.1580
- R2 Score: 0.7907


Lasso
Model performance for Training set
- Root Mean Squared Error: 344703.1931
- Mean Absolute Error: 167199.0444
- R2 Score: 0.8535
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 401575.7318
- Mean Absolute Error: 180013.6215
- R2 Score: 0.7858


Ridge
Model performance for Training set
- Root Mean Squared Error: 395811.4554
- Mean Absolute Error: 185942.2017
- R2 Score: 0.8068
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 408638.8095
- Mean Absolute Error: 197097.5191
- R2 Score: 0.7782


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 319478.3611
- Mean 

In [16]:
# Hyperparameter tuning
knn_params = {"n_neighbors":[2,3,10,20,50]}
rf_params = {
    "max_depth":[5,8,15,None,10],
    "max_features":[5,7,"auto",8],
    "min_samples_split":[2,8,15,20],
    "n_estimators":[100,200,500,1000]
}

In [17]:
# Models list for Hyperparameter tuning
randomcv_models = [
    ('KNN', KNeighborsRegressor(), knn_params),
    ('RF', RandomForestRegressor(), rf_params)
]

In [18]:
from sklearn.model_selection import RandomizedSearchCV
model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model, param_distributions=params,
     n_iter=100,
     cv=3,
     verbose=2,
     n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"--------------------Best Params for {model_name}-------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
--------------------Best Params for KNN-------------------
{'n_neighbors': 10}
--------------------Best Params for RF-------------------
{'n_estimators': 200, 'min_samples_split': 2, 'max_features': 8, 'max_depth': None}


In [19]:
models={
    "K-Neighbors Regressor": KNeighborsRegressor(n_neighbors=10, n_jobs=-1),
    "Random Forest": RandomForestRegressor(n_estimators= 200, min_samples_split=2, max_features=8, max_depth=None, n_jobs=-1)
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')

    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))

    print('='*35)
    print('\n')

K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 357601.1213
- Mean Absolute Error: 97864.4225
- R2 Score: 0.8423
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 250400.8173
- Mean Absolute Error: 108817.7181
- R2 Score: 0.9167


Random Forest
Model performance for Training set
- Root Mean Squared Error: 142619.2920
- Mean Absolute Error: 39715.2968
- R2 Score: 0.9749
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 203362.3386
- Mean Absolute Error: 97895.8603
- R2 Score: 0.9451




In [16]:
models={
    "AdaBoost Regressor": AdaBoostRegressor()
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')

    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))

    print('='*35)
    print('\n')

AdaBoost Regressor
Model performance for Training set
- Root Mean Squared Error: 444050.5567
- Mean Absolute Error: 340994.4213
- R2 Score: 0.7569
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 476626.9030
- Mean Absolute Error: 358499.9688
- R2 Score: 0.6982




In [17]:
# Hyperparameter tuning
ada_params = {
    "n_estimators": [50,60,70,80],
    "loss":['linear','square','exponential'],
}
# Models list for Hyperparameter tuning
randomcv_models = [
    ('AB', AdaBoostRegressor(), ada_params),
]

In [18]:
from sklearn.model_selection import RandomizedSearchCV
model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model, param_distributions=params,
     n_iter=100,
     cv=3,
     verbose=2,
     n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"--------------------Best Params for {model_name}-------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 12 candidates, totalling 36 fits
--------------------Best Params for AB-------------------
{'n_estimators': 50, 'loss': 'exponential'}


In [19]:
models={
    "AdaBoost Regressor": AdaBoostRegressor(n_estimators= 50, loss='exponential'),
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')

    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))

    print('='*35)
    print('\n')

AdaBoost Regressor
Model performance for Training set
- Root Mean Squared Error: 464495.6370
- Mean Absolute Error: 376967.4431
- R2 Score: 0.7340
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 490649.7112
- Mean Absolute Error: 392214.9812
- R2 Score: 0.6802


